In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time  #
import pandas as pd
from tqdm import tqdm
import openai
!pip install pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, LpStatusOptimal



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 18.4 MB/s eta 0:00:00


In [ ]:
#global variables

#chat GPT key token
key ='YOUR_API_KEY'


calorie_goal = 3400 #daily goal
protein_goal = 270
carbs_goal = 410
fats_goal = 80
budget = 90 #weekly budget
zip_code = "Your_ZIP_CODE_HERE"



In [ ]:
# Define your list of grocery items
grocery_list = [
    "boneless skinless chicken breast",
    "brown rice",
    "whole milk",
    "frozen broccoli",
    "Salmon",
    "Quinoa",
    "Eggs",
    "Kodiak Cakes protein pancake mix",
    'Greek yogurt',
    'wheat bread',
    'sweet potatoe',
    'wholly guacamole',
    'peanut butter',
    'honey',
    'canned tuna',
    'coottage cheese',
    'shrimp',
    'ground turkey',
    'lentils',
    'chickpeas',
    'edamame',
    'whey protein powder',
    'Kodiak protein oats',
    'frozen butternut squash',
    'frozen bananas',
    'apple slices',
    'frozen berries',
    'black beans',
    'kidney beans',
    'whole-wheat pasta',
    'olive oil',
    'coconut oil',
    'hummus',
    'turkey breast',
    'sirloin steak',
    'pork tenderloin',
    'frozen green peas',
    'baby cut carrots',
    'yukon potatoes',
    'goat cheese',
    'mozzarella string cheese',
    'Bibigo cooked white rice bowls',
    "thai red currry paste",



]

In [ ]:
def get_item_data(items, zip_code):

    grocery_dict = {}
    # Set the API key here
    openai.api_key = key

    for item in tqdm(items, desc="Fetching item data"): #added tqdm here
        prompt = f"What is the price for the first result you find on the internet when you search: {item} at Walmart, in the zip {zip_code}, please list out for me in a dictionary for python use, the Quantity, Fats, Carbs, Protein, Price, and Calories per serving, and Number of Servings of this item and nothing else."

        # Initialize the OpenAI client - No longer needed
        # client = openai.OpenAI() #This line is no longer needed as the OpenAI client is now initialized with the key.

        # Use the client to make the API call
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )

        # Assuming response is in the correct format, store it in the dictionary
        grocery_dict[item] = response

    return grocery_dict



#grocery_info = get_item_data(grocery_list, zip_code)

# SAVE OUTPUT AS JSON FILE
# with open('data.json', 'w') as json_file:
#     json.dump(grocery_info, json_file, indent=4)

In [ ]:
#Open and load saved json data from previous step
with open('scrapped_data.json', 'r') as json_file:
    scrapped_data = json.load(json_file)

In [ ]:
scrapped_data

{'90% ground beef': {'Quantity': '16 oz',
  'Price': '$7.48',
  'Fats': '11 g',
  'Carbs': '0 g',
  'Protein': '22 g',
  'Calories': '200',
  'Servings': '4'},
 'brown rice': {'Quantity': '32 oz',
  'Price': '$1.64',
  'Fats': '1.5 g',
  'Carbs': '32 g',
  'Protein': '3 g',
  'Calories': '150',
  'Servings': '21'},
 'boneless skinless chicken breast': {'Quantity': '112 g',
  'Price': '$7.96',
  'Fats': '4 g',
  'Carbs': '0 g',
  'Protein': '25 g',
  'Calories': '140',
  'Servings': '12'},
 'whole milk': {'Quantity': '3785 ml',
  'Price': '$3.22',
  'Fats': '8 g',
  'Carbs': '12 g',
  'Protein': '78 g',
  'Calories': '150',
  'Servings': '16'},
 'frozen broccoli': {'Quantity': '12 oz',
  'Price': '$1.16',
  'Fats': '0 g',
  'Carbs': '5 g',
  'Protein': '3 g',
  'Calories': '30',
  'Servings': '4'},
 'Salmon': {'Quantity': '16 oz',
  'Price': '$9.92',
  'Fats': '18 g',
  'Carbs': '0 g',
  'Protein': '23 g',
  'Calories': '250',
  'Servings': '4'},
 'Quinoa': {'Quantity': '16 oz',
  'Pric

In [ ]:
food_df = pd.DataFrame.from_dict(data, orient='index')
food_df.reset_index(inplace=True)
food_df.rename(columns={'index': 'Item'}, inplace=True)
food_df['breakfast'] = food_df['Item'].apply(lambda x: 1 if x in ['frozen berries',
                                                        'frozen bananas',
                                                        'Eggs',
                                                        'Kodiak Cakes protein pancake mix',
                                                        'Greek Yogurt', 'Kodiak protein oats',
                                                        'frozen bananas',
                                                        ] else 0)
food_df.head()

,Item,Quantity,Price,Fats,Carbs,Protein,Calories,Servings,breakfast
0,90% ground beef,16 oz,$7.48,11 g,0 g,22 g,200,4,0
1,brown rice,32 oz,$1.64,1.5 g,32 g,3 g,150,21,0
2,boneless skinless chicken breast,112 g,$7.96,4 g,0 g,25 g,140,12,0
3,whole milk,3785 ml,$3.22,8 g,12 g,78 g,150,16,0
4,frozen broccoli,12 oz,$1.16,0 g,5 g,3 g,30,4,0


In [ ]:
food_df.columns

Index(['Item', 'Quantity', 'Price', 'Fats', 'Carbs', 'Protein', 'Calories',
       'Servings', 'breakfast'],
      dtype='object')

In [ ]:
#Clean df
def clean_df(df):
    df['Price'] = df['Price'].apply(lambda x: float(x.replace('$', '')))
    df['Calories'] = df['Calories'].apply(lambda x: int(x))
    df['Carbs'] = df['Carbs'].apply(lambda x: int(x.replace('g', '')))
    df['Protein'] = df['Protein'].apply(lambda x: float(x.replace('g', '')))
    df['Fats'] = df['Fats'].apply(lambda x: float(x.replace('g', '')))
    df['Servings'] = df['Servings'].apply(lambda x: float(x))
    df['Unit'] = df['Quantity'].apply(lambda x: x.split(' ')[-1])
    df['Quantity'] = df['Quantity'].apply(lambda x: float(x.split(' ')[0]))
    df['Calories'] = df['Calories'].apply(lambda x: float(x))
    return df

food_df = clean_df(food_df)
food_df.head()

,Item,Quantity,Price,Fats,Carbs,Protein,Calories,Servings,breakfast,Unit
0,90% ground beef,16.0,7.48,11.0,0,22.0,200.0,4.0,0,oz
1,brown rice,32.0,1.64,1.5,32,3.0,150.0,21.0,0,oz
2,boneless skinless chicken breast,112.0,7.96,4.0,0,25.0,140.0,12.0,0,g
3,whole milk,3785.0,3.22,8.0,12,78.0,150.0,16.0,0,ml
4,frozen broccoli,12.0,1.16,0.0,5,3.0,30.0,4.0,0,oz


In [ ]:

def meal_plan_result(df, calorie_goal, protein_goal, carbs_goal, fats_goal, budget):
    """
    Creates an optimized meal plan using linear programming, prioritizing the use of more ingredients while staying within budget.

    Args:
        df: DataFrame containing food item data.
        calorie_goal: Daily calorie goal.
        protein_goal: Daily protein goal.
        carbs_goal: Daily carbs goal.
        fats_goal: Daily fats goal.
        budget: Weekly budget.

    Returns:
        A dictionary containing the optimized meal plan and its total cost.
    """

    # Create a LP problem
    prob = LpProblem("MealPlan", LpMaximize)

    # Define decision variables (servings of each food item for each meal)
    meal_names = ["breakfast", "lunch", "dinner"]
    food_vars = LpVariable.dicts("Food", [(food, meal) for food in df.index for meal in meal_names], lowBound=0, cat='Integer')

    # Binary variables to track if a food item is used
    food_used = LpVariable.dicts("FoodUsed", df.index, cat='Binary')

    # Objective function: Maximize the number of unique ingredients used
    prob += lpSum([food_used[i] for i in df.index]), "MaximizeIngredients"

    # Breakfast food requirement: Enforce that only breakfast foods can be selected for breakfast
    for food_index, meal in food_vars:
        if meal == 'breakfast' and df.loc[food_index, 'breakfast'] == 0:
            prob += food_vars[(food_index, meal)] == 0, f"BreakfastConstraint_{food_index}_{meal}"

    # Link the binary variables to the food_vars (if a food is used in any meal, food_used = 1)
    for food_index in df.index:
        prob += lpSum(food_vars[(food_index, meal)] for meal in meal_names) >= food_used[food_index], f"FoodUsageConstraint_{food_index}"

    # Constraints:
    # Daily macro goals
    prob += lpSum([df.loc[i, 'Calories'] * food_vars[(i, meal)] for i in df.index for meal in meal_names]) >= calorie_goal
    prob += lpSum([df.loc[i, 'Protein'] * food_vars[(i, meal)] for i in df.index for meal in meal_names]) >= protein_goal
    prob += lpSum([df.loc[i, 'Carbs'] * food_vars[(i, meal)] for i in df.index for meal in meal_names]) >= carbs_goal
    prob += lpSum([df.loc[i, 'Fats'] * food_vars[(i, meal)] for i in df.index for meal in meal_names]) >= fats_goal

    # Budget constraint
    prob += lpSum([df.loc[i, 'Price'] * food_vars[(i, meal)] for i in df.index for meal in meal_names]) <= budget

    # Serving size constraint (max 4 servings per item per meal)
    for food_index, meal in food_vars:
        prob += food_vars[(food_index, meal)] <= 4, f"ServingSizeConstraint_{food_index}_{meal}"

    # Prevent repetition of food items across meals within the same day
    for food_index in df.index:
        prob += lpSum(food_vars[(food_index, meal)] for meal in meal_names) <= 1, f"RepetitionConstraint_{food_index}"

    # Requirement for at least one item per meal
    for meal in meal_names:
        prob += lpSum(food_vars[(food_index, meal)] for food_index in df.index) >= 1, f"MealRequirement_{meal}"

    # Solve the problem
    prob.solve()

    # Extract the meal plan and total cost
    meal_plan = {"Breakfast": {}, "Lunch_Dinner": {}}
    total_cost = 0
    for food_index, meal in food_vars:
        if food_vars[(food_index, meal)].varValue > 0:
            if meal == "breakfast":
                meal_plan["Breakfast"][df.loc[food_index, 'Item']] = food_vars[(food_index, meal)].varValue
            else:
                meal_plan["Lunch_Dinner"][df.loc[food_index, 'Item']] = food_vars[(food_index, meal)].varValue
            total_cost += df.loc[food_index, 'Price'] * food_vars[(food_index, meal)].varValue

    # Calculate the number of unique ingredients used and convert to integer
    unique_ingredients_used = int(sum(food_used[i].varValue for i in df.index))

    return {"meal_plan": meal_plan, "total_cost": total_cost, "unique_ingredients_used": unique_ingredients_used}



In [71]:
food_to_buy = meal_plan_result(food_df, calorie_goal, protein_goal, carbs_goal, fats_goal, 90)
print(food_to_buy)

{'meal_plan': {'Breakfast': {'Eggs': 1.0, 'Kodiak Cakes protein pancake mix': 1.0, 'Kodiak protein oats': 1.0, 'frozen bananas': 1.0}, 'Lunch_Dinner': {'brown rice': 1.0, 'whole milk': 1.0, 'frozen broccoli': 1.0, 'Quinoa': 1.0, 'Greek yogurt': 1.0, 'wheat bread': 1.0, 'sweet potatoe': 1.0, 'wholly guacamole': 1.0, 'peanut butter': 1.0, 'canned tuna': 1.0, 'coottage cheese': 1.0, 'ground turkey': 1.0, 'lentils': 1.0, 'chickpeas': 1.0, 'edamame': 1.0, 'frozen butternut squash': 1.0, 'frozen berries': 1.0, 'black beans': 1.0, 'kidney beans': 1.0, 'whole-wheat pasta': 1.0, 'frozen green peas': 1.0, 'baby cut carrots': 1.0, 'yukon potatoes': 1.0, 'goat cheese': 1.0, 'mozzarella string cheese': 1.0}}, 'total_cost': 89.71000000000001, 'unique_ingredients_used': 29}


In [ ]:
def get_total_cals(meeal_dict):
    total_cals = 0
    total_fats = 0
    total_carbs = 0
    total_protein = 0

    for i in meeal_dict['meal_plan'].keys():
        for j in meeal_dict['meal_plan'][i].keys():
            total_cals += food_df[food_df['Item'] == j]['Calories'].values[0] * meeal_dict['meal_plan'][i][j]
            total_fats += food_df[food_df['Item'] == j]['Fats'].values[0] * meeal_dict['meal_plan'][i][j]
            total_carbs += food_df[food_df['Item'] == j]['Carbs'].values[0] * meeal_dict['meal_plan'][i][j]
            total_protein += food_df[food_df['Item'] == j]['Protein'].values[0] * meeal_dict['meal_plan'][i][j]

    meal_plan_totals = {"total_cals": total_cals,
                        "total_fats": total_fats,
                        "total_carbs": total_carbs,
                        "total_protein": total_protein}

    return meal_plan_totals




In [ ]:
food_totals = get_total_cals(food_to_buy)
food_totals

{'total_cals': 3415.0,
 'total_fats': 98.5,
 'total_carbs': 435.0,
 'total_protein': 281.6}

In [ ]:
def get_recipe_recommendations_and_missing_ingredients(purchased_foods):
    # Construct the prompt
    foods_str = ", ".join([f"{item}: {quantity}" for item, quantity in purchased_foods.items()])
    prompt = f"Given this dictionary of foods I have bought and their respective quantities: {{{foods_str}}}, please give me some recommendations for recipes, and also identify any missing ingredients I would need."

    try:
        # Call the OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )

        # Extract the response text
        extra_ingredients = response['choices'][0]['message']['content'].strip()
        return extra_ingredients

    except Exception as e:
        return f"An error occurred: {str(e)}"

In [ ]:
#extra_ingredients = get_recipe_recommendations_and_missing_ingredients(food_to_buy)
# def save_string_to_file(string, filename):
#     with open(filename, 'w') as file:
#         file.write(string)

# save_string_to_file(extra_ingredients, 'extra_ingredients.txt')

In [62]:
def load_string_from_file(filename):
    with open(filename, 'r') as file:
        return file.read()
extra = load_string_from_file('extra_ingredients.txt')

In [64]:
print(extra)

 
    Based on the ingredients you've listed, here are a few recipe ideas along with any additional ingredients you may need for each recipe:

1. Shrimp and Broccoli Stir-Fry
Ingredients Needed:

Your Ingredients: Shrimp, frozen broccoli, brown rice (or quinoa), hummus (optional for dipping).
Additional Ingredients: Soy sauce, garlic, ginger, olive oil or sesame oil, red pepper flakes (optional).
Instructions:

Sauté minced garlic and ginger in oil.
Add shrimp and cook until pink.
Toss in broccoli and cook until tender.
Serve over brown rice or quinoa, drizzled with soy sauce.
2. Quinoa Salad with Chickpeas and Guacamole
Ingredients Needed:

Your Ingredients: Quinoa, chickpeas, wholly guacamole, frozen green peas, baby cut carrots, goat cheese.
Additional Ingredients: Lemon juice, olive oil, salt, pepper, diced tomatoes or bell peppers (optional).
Instructions:

Cook quinoa as per package instructions.
In a large bowl, combine cooked quinoa, chickpeas, green peas, chopped carrots, and 